## Loading LIbraries

In [5]:
import pandas as pd
import seaborn as sns
from elo import rate_1vs1

## Loading Match Data

In [7]:
matches=pd.read_csv('MatchData.csv',low_memory=False)  ## IDk what low memory does but it works



### Getting List of all teams

In [8]:

## List of all teams
teamset=set()
for team in matches['HomeTeam']:
    teamset.add(team)
for team in matches['AwayTeam']:
    teamset.add(team)
print(teamset)

{nan, 'Granada', 'Valencia', 'Malaga', 'Eibar', 'Girona', 'Espanol', 'Getafe', 'Numancia', 'La Coruna', 'Ath Madrid', 'Las Palmas', 'Recreativo', 'Cordoba', 'Huesca', 'Alaves', 'Santander', 'Extremadura', 'Ath Bilbao', 'Zaragoza', 'Villarreal', 'Sociedad', 'Elche', 'Levante', 'Betis', 'Celta', 'Compostela', 'Real Madrid', 'Lerida', 'Logrones', 'Barcelona', 'Sp Gijon', 'Vallecano', 'Merida', 'Hercules', 'Almeria', 'Osasuna', 'Gimnastic', 'Valladolid', 'Mallorca', 'Murcia', 'Tenerife', 'Albacete', 'Oviedo', 'Sevilla', 'Cadiz', 'Leganes', 'Salamanca', 'Xerez', 'Villareal'}


TeamData of each and every team( We will add fields as we go on)

In [39]:
teamsdata={}
for team in teamset:
    teamdata={}
    teamdata['matches_played']=0
    teamdata['won']=0
    teamdata['tie']=0
    teamdata['elo']=1200  #Default rating
    teamsdata[team]=teamdata

In [10]:
matches.describe()

,Unnamed: 0,FTHG,FTAG,Season,HTHG,HTAG,GBH,GBD,GBA,IWH,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
count,11051.000000,10355.000000,10355.000000,11051.000000,9593.000000,9593.000000,4729.000000,4729.000000,4729.000000,7509.000000,...,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000
mean,208.964890,1.568904,1.109512,2005.249842,0.693214,0.483061,2.355394,3.598336,4.379975,2.438172,...,1.863664,-0.316194,1.955792,1.958629,1.959669,1.962530,2.018463,2.018936,1.937470,1.940449
std,129.742154,1.318418,1.117745,8.097132,0.843592,0.696223,1.424818,0.942945,3.087454,1.665292,...,0.455824,0.740895,0.102628,0.104099,0.103082,0.103994,0.104796,0.109858,0.097723,0.097881
min,0.000000,0.000000,0.000000,1993.000000,0.000000,0.000000,1.050000,2.350000,1.120000,1.030000,...,1.290000,-2.750000,1.680000,1.670000,1.700000,1.750000,1.780000,1.800000,1.710000,1.750000
25%,100.000000,1.000000,0.000000,1998.000000,0.000000,0.000000,1.700000,3.200000,2.750000,1.650000,...,1.550000,-0.750000,1.880000,1.880000,1.880000,1.880000,1.940000,1.930000,1.860000,1.860000
50%,203.000000,1.000000,1.000000,2005.000000,0.000000,0.000000,2.050000,3.300000,3.500000,2.000000,...,1.740000,-0.250000,1.960000,1.970000,1.950000,1.970000,2.010000,2.020000,1.930000,1.940000
75%,306.000000,2.000000,2.000000,2012.000000,1.000000,1.000000,2.450000,3.500000,4.800000,2.500000,...,2.040000,0.000000,2.040000,2.040000,2.040000,2.045000,2.100000,2.105000,2.015000,2.020000
max,551.000000,10.000000,8.000000,2020.000000,6.000000,6.000000,17.000000,11.000000,34.000000,20.000000,...,4.130000,1.750000,2.160000,2.190000,2.200000,2.290000,2.260000,2.300000,2.150000,2.210000


In [46]:
for index, row in matches.iterrows(): 
    ht=row['HomeTeam']
    teamsdata[ht]['matches_played']+=1
    
    at=row["AwayTeam"]
#     print(ht ,at)
    teamsdata[at]['matches_played']+=1
    result=row['FTR']
#     print(result)
    if(result=='D'):
#         print("Tie")
        teamsdata[ht]['tie']+=1
        teamsdata[at]['tie']+=1
        wer=teamsdata[ht]['elo']
        ler=teamsdata[at]['elo']
        nwer , nler = rate_1vs1(wer,ler,drawn=True)
        teamsdata[ht]['elo']=nwer
        teamsdata[at]['elo']=nler
    else:
        
        winner=ht if result=='H' else at
        loser=at if result=='H' else ht
#         print("Winner",winner)
        teamsdata[winner]['won']+=1
        ## Updata elo ratings
        wer=teamsdata[winner]['elo']
        ler=teamsdata[loser]['elo']
        nwer , nler = rate_1vs1(wer,ler)
        teamsdata[winner]['elo']=nwer
        teamsdata[loser]['elo']=nler
        

teamsdata

{'Alaves': {'elo': 1200.1287207782807,
  'matches_played': 770,
  'tie': 172,
  'won': 258},
 'Albacete': {'elo': 1153.366268514086,
  'matches_played': 369,
  'tie': 116,
  'won': 94},
 'Almeria': {'elo': 1143.3352592542162,
  'matches_played': 456,
  'tie': 112,
  'won': 124},
 'Ath Bilbao': {'elo': 1252.4961550183923,
  'matches_played': 2053,
  'tie': 572,
  'won': 761},
 'Ath Madrid': {'elo': 1415.9123501465199,
  'matches_played': 1901,
  'tie': 468,
  'won': 897},
 'Barcelona': {'elo': 1503.873783700005,
  'matches_played': 2053,
  'tie': 414,
  'won': 1315},
 'Betis': {'elo': 1224.7134778285815,
  'matches_played': 1676,
  'tie': 472,
  'won': 574},
 'Cadiz': {'elo': 1182.5187406069622,
  'matches_played': 86,
  'tie': 26,
  'won': 20},
 'Celta': {'elo': 1206.6617077775104,
  'matches_played': 1623,
  'tie': 445,
  'won': 562},
 'Compostela': {'elo': 1190.1820536264206,
  'matches_played': 320,
  'tie': 90,
  'won': 104},
 'Cordoba': {'elo': 1132.1541546490569,
  'matches_playe

In [50]:
teamstable=pd.DataFrame.from_dict(teamsdata,orient='Index')
teamstable['Team']=teamstable.index
teamstable.sort_values(by=['elo'],inplace=True,ascending=False)
topteams=teamstable.head(8)
topteams

,matches_played,won,tie,elo,Team
Barcelona,2053,1315,414,1503.873784,Barcelona
Real Madrid,2056,1292,392,1461.179549,Real Madrid
Ath Madrid,1901,897,468,1415.912350,Ath Madrid
Sevilla,1824,778,440,1347.593982,Sevilla
Villarreal,1430,602,372,1297.289494,Villarreal
Valencia,2054,948,502,1292.175058,Valencia
Sociedad,1825,662,494,1272.018088,Sociedad
Getafe,1126,380,290,1266.070782,Getafe


In [52]:
sns.catplot(topteams, x="Team",y="elo")

/home/gagan/.local/lib/python3.6/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


ValueError: Could not interpret input 'elo'